[![imagenes](imagenes/pythonista.png)](https://pythonista.io)

# Introducción a Redes y HTTP en la Biblioteca Estándar

Aunque en el día a día usamos bibliotecas de terceros como `requests` o `httpx` (para clientes) y `FastAPI` o `Django` (para servidores), Python incluye todo lo necesario para construir internet desde cero.

## Módulos clave
*   `socket`: La interfaz de red de bajo nivel (TCP/UDP, DNS).
*   `http.server`: Un servidor web básico implementado en Python puro.
*   `urllib.request`: Cliente HTTP histórico (algo verboso, pero estándar).

## 1. Sockets: La tubería de Internet

Un socket es un punto final (endpoint) en una comunicación de red bidireccional. Todo el tráfico web (HTTP, FTP, Email) corre sobre sockets.

In [ ]:
import socket

# 1. Resolución de DNS (Domain Name System)
hostname = "python.org"
try:
    ip_address = socket.gethostbyname(hostname)
    print(f"El servidor de {hostname} está en la IP: {ip_address}")
except socket.gaierror:
    print("No se pudo resolver el nombre de dominio")

In [ ]:
# 2. Conectarse manualmente a un servidor HTTP (Puerto 80)
# Esto es lo que hacen los navegadores por debajo
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(("python.org", 80))

# Enviamos una petición HTTP cruda en bytes
request = b"GET / HTTP/1.1\r\nHost: python.org\r\n\r\n"
s.sendall(request)

# Recibimos la respuesta (primeros 500 bytes)
response = s.recv(500)
print("--- Respuesta del Servidor (Raw Bytes) ---")
print(response.decode('utf-8'))
s.close()

## 2. Cliente HTTP: `urllib.request`

Python ofrece una abstracción sobre los sockets para hablar HTTP. Es más complejo que `requests.get()`, pero viene preinstalado en todas partes.

In [ ]:
import urllib.request
import json

url = "https://httpbin.org/json"

print(f"Consultando {url}...")
with urllib.request.urlopen(url) as response:
    print(f"Estado: {response.status}")
    data = response.read()
    
    # Decodificamos JSON
    json_data = json.loads(data)
    print("Datos recibidos:", json_data['slideshow']['author'])

## 3. Servidor Web Mínimo: `http.server`

Python puede servir archivos estáticos con una sola línea de comando:
`python -m http.server 8000`

También podemos hacerlo desde código para entender el ciclo Request-Response.

In [ ]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import threading

# Definimos cómo responder a las peticiones
class SimpleHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        self.send_response(200)
        self.send_header("Content-type", "text/plain")
        self.end_headers()
        self.wfile.write(b"Hola desde Python Standard Library!")

# Ejecutamos el servidor en un hilo aparte para no bloquear el cuaderno
def run_server():
    server = HTTPServer(("localhost", 8081), SimpleHandler)
    print("Servidor corriendo en http://localhost:8081")
    # server.serve_forever() # Comentado para no bloquear eternamente
    server.handle_request() # Solo maneja una petición y termina

t = threading.Thread(target=run_server)
t.start()

In [ ]:
# Hacemos una petición a nuestro propio servidor
import time
time.sleep(1) # Esperamos a que inicie

with urllib.request.urlopen("http://localhost:8081") as response:
    print("Respuesta local:", response.read().decode('utf-8'))

t.join()

<p style="text-align: center"><a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Licencia Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/80x15.png" /></a><br />Esta obra está bajo una <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Licencia Creative Commons Atribución 4.0 Internacional</a>.</p>
<p style="text-align: center">&copy; José Luis Chiquete Valdivieso. 2017-2026.</p>